# koBERT fine-tuning

In [ ]:
! pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

In [ ]:
! pip install transformers

In [ ]:
! pip install -r /content/drive/MyDrive/cp1/requirements.txt

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/cp1/description.tsv', sep='\t')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,id,description,title,genre
0,58289261,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다. 군부를...",남주들의 집착보다 내 탈영이 빠르겠다,로판
1,55275635,[카카오 독점 연재] 혈마(血魔) 천무정. 피도 눈물도 없는 무림 최강의 존재. ...,마법황가 무공천재,판타지
2,52716578,"시골 객점의 아들 송우문. 어릴 적부터 몸이 약했던 그는, 어느 날 노도사가 그려 ...",불패검선,무협
3,58404017,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다. 결혼을 6번 ...,7번의 결혼이 예정되어 있습니다,로판
4,47050646,공사장에서 벽돌 나르고 삽질하는 불운한 인생의 신영우. 그런데 심지어 게임 속에서 ...,템빨,판타지


In [6]:
# 장르 종류
df['genre'].unique()

array(['로판', '판타지', '무협', '현판', '로맨스'], dtype=object)

In [7]:
romance_fantasy = df[df['genre']=='로판'][['title', 'description']]
fantasy = df[df['genre']=='판타지'][['title', 'description']]
real_fantasy = df[df['genre']=='현판'][['title', 'description']]
martial = df[df['genre']=='무협'][['title', 'description']]
romance = df[df['genre']=='로맨스'][['title', 'description']]

fantasy_all = pd.concat([romance_fantasy, fantasy, real_fantasy])
fantasy_all

,title,description
0,남주들의 집착보다 내 탈영이 빠르겠다,"""너는 오래 전 죽은 그 애를 참 닮았군.""의 '그 애' 역할로 빙의했다. 군부를..."
3,7번의 결혼이 예정되어 있습니다,아가사 노빌리스는 몇 세기 만에 나온 어마어마한 자산의 상속녀였다. 결혼을 6번 ...
9,이 3세는 악역입니다,‘빙의물에 빙의를 시키는 게 어디 있어...!!’ <빙의했는데 흑막의 손녀였다>라...
13,정의로운 미친개의 며느리가 되었다,시가인 로지문드 공작가에 착취당하다 재산을 모두 빼앗기고 죽은 이사벨라. 기적처럼 ...
34,시한부 엑스트라의 시간,"가문의 후계자도, 귀여움을 받는 쌍둥이 막냇동생도 되지 못한 어중간한 둘째로, 평생..."
...,...,...
4041,뉴욕 사는 그 마법사,내 이름이 뭐였지? 왜 아무것도 떠오르질 않는 거지? 대체 내가 왜 이런 음침한 ...
4054,아티팩트 메이지,내가 제일 잘 하는 게 뭔 줄 알아? 아티팩트 만드는 거야. 무기도 좋고 방어구도 ...
4060,역대 최강의 매니저,"[독점연재] - 부, 명예, 권력, 힘. 200년 전의 약속을 따라 모든 것을 이..."
4069,에드가&재호,"뮤지컬 배우를 꿈꾸는 성악과 대학생 신재호. 매일 밤 그는, 레무리아 대륙의 사냥..."


In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(fantasy_all, test_size=0.1, random_state=10)

In [ ]:
train_data

,title,description
236,나의 악당들,"평범한 서른 살의 직장인, 김승수. 어느 날, 오랫동안 플레이해 온 게임 '다크월드..."
1609,황제의 청혼(단독 선공개),"스물셋의 젊은 황제 ‘칼하트’는 훌륭한 군주이다. 냉정하고 오만하지만, 잘생긴 외모..."
3997,붉은 물빛의 레이디,"연애 한 번 못 해본 내 인생에, 갑자기 남편이 나타났다. 그것도 살벌하기 짝이 없..."
29,천재 궁수의 스트리밍,"선수권 대회 최연소 우승, 양궁 유망주. 승승장구 하는 줄 알았으나 비운의 사고로..."
3125,배드 엔딩 메이커,"“사표 쓰세요, 민철 씨.” 경영진의 과금 정책에 반대한다고, 청춘을 바쳤던 직장..."
...,...,...
3685,"공작가의 수치, 회귀하다",세계의 부와 권력 그리고 헌터들 이제 내가 그 모든 걸 결정한다! 마지막 결전에 ...
792,이차원 용병,연재 시작 즉시 절대적 지지를 받은 화제의 작품! 『이차원 용병』 취직하고 싶습...
1433,이세계 소환사는 프로게이머,최고의 프로게이머였지만 이제는 퇴물 소리를 듣는 이신우. 그냥 게임 속에서 살고 싶...
1664,우리 아내가 달라졌어요,완벽한 남자의 완벽한 아내가 되고 싶었던 안젤리카. 긴 짝사랑 끝에 소공작과의 결혼...


In [ ]:
train_data.to_csv('/content/drive/MyDrive/cp1/data/train.tsv', sep='\t', index=False)
test_data.to_csv('/content/drive/MyDrive/cp1/data/test.tsv', sep='\t', index=False)

# train model

In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-base-v2')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/496M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/682k [00:00<?, ?B/s]

In [ ]:
! pip install torchtext==0.8.0 torch==1.7.1 pytorch-lightning==1.2.2

  Using cached torch-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (776.8 MB)
  Using cached pytorch_lightning-1.2.2-py3-none-any.whl (816 kB)
  Using cached PyYAML-6.0-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (596 kB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0
    Uninstalling torch-1.10.0:
      Successfully uninstalled torch-1.10.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 5.4.1
    Uninstalling PyYAML-5.4.1:
      Successfully uninstalled PyYAML-5.4.1
  Attempting uninstall: pytorch-lightning
    Found existing installation: pytorch-lightning 1.3.8
    Uninstalling pytorch-lightning-1.3.8:
      Successfully uninstalled pytorch-lightning-1.3.8
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you h

In [ ]:
! python /content/drive/MyDrive/cp1/train.py  --gradient_clip_val 1.0 --max_epochs 3 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='logs', deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=3, max_len

In [ ]:
!pip install pyyaml==5.4.1

  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytorch-lightning 1.2.2 requires PyYAML!=5.4.*,>=5.1, but you have pyyaml 5.4.1 which is incompatible.
kobart 0.4 requires pytorch-lightning==1.2.1, but you have pytorch-lightning 1.2.2 which is incompatible.
kobart 0.4 requires transformers==4.3.3, but you have transformers 4.8.2 which is incompatible.


In [ ]:
! python /content/drive/MyDrive/cp1/get_model_binary.py --hparams /content/drive/MyDrive/cp1/logs/tb_logs/default/version_0/hparams.yaml --model_binary /content/drive/MyDrive/cp1/logs/model_chp/epoch=01-val_loss=2.935.ckpt


In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/cp1/kobart_summary1')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

In [ ]:
text = test_data['description'][128]

In [ ]:
text = """
공포 게임 <살인자들의 밤> 속 리어먼드 공작가의 가정교사인 ‘아르비체’로 빙의했다. 하지만 아르비체는 게임 속 열 번째 희생자가 될 인물. 역시나 빙의하자마자 범인에게 목숨을 잃는다.
다시 눈을 뜨니 상단바의 하트가 하나 사라졌다? 남은 하트는 겨우 두 개. 아르비체가 원하는 것은 단 하나, 살아남는 것. 1회차는 실패했지만 2회차에서는 무조건 쥐 죽은 듯 지내야 한다.
유력한 용의자는 그녀의 고용주인 리어먼드 공작, 레이커스!
그가 범인이라고 생각해 최대한 그와 멀어지려는 아르비체의 행동은 의도와 달리 레이커스의 흥미를 끌고야 마는데...

  “제가 그린 양이 흥미롭다고 말했던 기억이 나나요? 그 말, 철회해야겠군요.”

듣던 중 반가운 소리였다. 살인마의 흥미라니, 끔찍하기만 하다.

  “흥미롭다는 말만으로는 부족할지도 모르겠군요.”  

[레이커스 리어먼드의 호감도가 상승했습니다.]

  '....어? 이게 아닌데?'
  
#착각물 #오해물 #로맨스릴러 #게임빙의물 #생존물
"""

# 정답 : 공포게임 조연인데요, 죽이지 마세요

In [ ]:
text = """
부모님이 돌아가셨다.
물려받은 재산으로 어찌저찌 빚은 면했지만 딸린 동생만 세 명이다!
가족끼리 찢어져 살 수도 없는 노릇이라 하녀 일을 해 어찌저찌 동생들을 잘 키워냈다.
집안도 다시 일어났겠다 이제 내 인생 찾아서 연애하려는데….

“누님. 그 남자는 버리십시오. 제 주변에 괜찮은 남자를 소개시켜 드리겠습니다.”

첫째 동생 에드워드가 제안했다. 내 동생이지만 피도 눈물도 없는 녀석이 에드워드다.
그런 녀석의 기준에 괜찮은 남자라면 얼마나 쓰레기일지 예상만으로도 무섭다.
그래서 소개팅 제의를 거절하자, 에드워드가 샐쭉하게 웃는다.

“누이. 그 남자 뒷조사를 해 보니까 빚이 오천만 베르크야.”

이어서… 둘째인 다니엘이 놀란 척 말했다.
얘, 다니엘. 그 빚 네가 지게 만들었잖아….

“누나. 누나! 그냥 결혼 같은 거 하지 말고 평생 우리랑 살자!”

그리 말하는 막내, 레오나르드의 옷에 피가 튀어 있었다.
내가 만나던 남자마다 막내를 만나고 나면 이상하게 나를 피하던데…
그리고 네 이름만 들어도 벌벌 떨던데… 아니지…?

난 분명 동생들을 곱게 키우려고 노력했는데… 어쩌다 이렇게 된 걸까?
"""

# 정답 : 흑막 남동생들이 내 연애를 방해한다

In [ ]:
import torch
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0)
output = model.generate(input_ids, eos_token_id=1, max_length=256, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
output

'내 동생에 괜찮은 남자를 소개시켜 드리겠습니다'

# 추가학습

In [ ]:
! python /content/drive/MyDrive/cp1/train.py  --gradient_clip_val 1.0 --max_epochs 3 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='logs', deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=3, max_len=512, max_steps=None, min_epochs=None, min_steps=None, model_path=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=4, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None, 

In [ ]:
!pip install pyyaml==5.4.1

In [ ]:
! python /content/drive/MyDrive/cp1/get_model_binary.py --hparams /content/drive/MyDrive/cp1/logs/tb_logs/default/version_0/hparams.yaml --model_binary /content/drive/MyDrive/cp1/logs/model_chp/epoch=01-val_loss=2.277.ckpt

/content/drive/MyDrive/cp1/get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)


In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/cp1/kobart_summary2')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

In [ ]:
import torch
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0)
output = model.generate(input_ids, eos_token_id=1, max_length=256, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
output

'내 동생에게 괜찮은 남자에게 소개팅을 거절합니다'

# 추가학습 2

In [ ]:
! python /content/drive/MyDrive/cp1/train.py  --gradient_clip_val 1.0 --max_epochs 10 --default_root_dir logs  --gpus 1 --batch_size 4 --num_workers 4

INFO:root:Namespace(accelerator=None, accumulate_grad_batches=1, amp_backend='native', amp_level='O2', auto_lr_find=False, auto_scale_batch_size=False, auto_select_gpus=False, automatic_optimization=None, batch_size=4, benchmark=False, check_val_every_n_epoch=1, checkpoint_callback=True, checkpoint_path=None, default_root_dir='logs', deterministic=False, distributed_backend=None, enable_pl_optimizer=None, fast_dev_run=False, flush_logs_every_n_steps=100, gpus=1, gradient_clip_val=1.0, limit_predict_batches=1.0, limit_test_batches=1.0, limit_train_batches=1.0, limit_val_batches=1.0, log_every_n_steps=50, log_gpu_memory=None, logger=True, lr=3e-05, max_epochs=10, max_len=512, max_steps=None, min_epochs=None, min_steps=None, model_path=None, move_metrics_to_cpu=False, multiple_trainloader_mode='max_size_cycle', num_nodes=1, num_processes=1, num_sanity_val_steps=2, num_workers=4, overfit_batches=0.0, plugins=None, precision=32, prepare_data_per_node=True, process_position=0, profiler=None,

In [ ]:
! python /content/drive/MyDrive/cp1/get_model_binary.py --hparams /content/drive/MyDrive/cp1/logs/tb_logs/default/version_0/hparams.yaml --model_binary /content/drive/MyDrive/cp1/model_chp/epoch=00-val_loss=2.623.ckpt

/content/drive/MyDrive/cp1/get_model_binary.py:13: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  hparams = yaml.load(f)
Downloading: 100% 4.00/4.00 [00:00<00:00, 2.44kB/s]
Downloading: 100% 111/111 [00:00<00:00, 38.5kB/s]
Downloading: 100% 682k/682k [00:00<00:00, 2.90MB/s]


In [ ]:
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained('/content/drive/MyDrive/cp1/kobart_summary3')
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-base-v2')

In [ ]:
import torch
input_ids = tokenizer.encode(text)
input_ids = torch.tensor(input_ids)
input_ids = input_ids.unsqueeze(0)
output = model.generate(input_ids, eos_token_id=1, max_length=256, num_beams=5)
output = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
output

'내 동생 잘 키워서 연애하려는데'

# Report

- 3 / 3 / 10회를 나누어 총 16 epochs 학습

- 우선 val_loss는 3을 넘지 않는 모델만 선정. 요약기의 성능은 수치만으로 판단하기는 어려우므로 test 데이터와 타 사이트에서 작품 소개 글을 직접 입력해보며 비교. 

  - 2회 학습 모델 : 핵심 문장을 가져오는 경향이 있으나 작명이라고 하기는 어려우나 나름대로 rough한 느낌이 있다.
  - 5회 학습 모델 : 2회 학습 모델보다 조금 더 정돈된 output
  - 7회 학습 모델 : input의 길이가 적당할 때 가장 제목다운 output